Что делать?

1.Датасет ml-latest

2.Вспомнить подходы, которые мы разбирали

3.Выбрать понравившийся подход к гибридным системам

4.Написать свою

In [12]:
from surprise import SVD, SVDpp
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import KNNWithMeans


import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

import pandas as pd
import numpy as np

from scipy.spatial.distance import correlation

In [2]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [3]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [4]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [28]:
#определяем топ фильмы отдельного пользователя
def ret_user_fav_films(userid, dataset):
    return dataset.loc[(dataset['userId'] == userid) & (dataset['rating'] == 5.0)].movieId.tolist()


#Создаем матрицу векторов с оценками для каждого фильма
def movie_vectorizer(dataset):
    qty_users = dataset.userId.unique().shape[0]
    movie_vector = {}

    for movie, group in dataset.groupby('movieId'):
        movie_vector[movie] = np.zeros(qty_users)
    
        for i in range(len(group.userId.values)):
            user = group.userId.values[i]
            rating = group.rating.values[i]
            movie_vector[movie][int(user - 1)] = rating
     
    return movie_vector  


#Первая рекомендация
def movies_recommend(my_fav_films, vector, size=10):
    full_movies_list = []
    for i in my_fav_films:
        titles = []
        distances = []
        for key in vector.keys():
            if key == i:
                continue
                
            titles.append(key)
            distances.append(correlation(vector[i], vector[key]))
        
        best_indexes = np.argsort(distances)[:size]
        best_movies = [(titles[z]) for z in best_indexes]
        full_movies_list.extend(best_movies)

    return list(set(full_movies_list) - set(my_fav_films))


#Вторая рекомендация

def make_rec(dataset, user_id, size=5):
    mov_id = []
    est = []
    for mov in dataset.iid.unique():
        mov_id.append(mov)
        est.append(algo.predict(uid=user_id, iid=mov).est)

        best_ind = np.argsort(est)[-size:]
        best_mov = [(mov_id[b]) for b in best_ind]
    return best_mov

In [29]:
#Запускаем систему:
user_films = ret_user_fav_films(20.0, movies_with_ratings)

movie_vector = movie_vectorizer(movies_with_ratings)

rec_movies_list = movies_recommend(user_films, movie_vector, 15)

user_rec_movies = movies_with_ratings.loc[movies_with_ratings.movieId.isin(rec_movies_list)]

In [30]:
#Подготовим данные для модели и обучим модель
dataset = pd.DataFrame({
    'uid': user_rec_movies.userId,
    'iid': user_rec_movies.movieId,
    'rating': user_rec_movies.rating
})

reader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))
data = Dataset.load_from_df(dataset, reader)

trainset = data.build_full_trainset()

algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [31]:

best_rec_mov = make_rec(dataset, 15)
best_rec_mov

[3147, 1196, 4720, 3606, 4380]

In [32]:


recommended_movies_for_user = movies.loc[movies.movieId.isin(best_rec_mov)]
recommended_movies_for_user

,movieId,title,genres
898,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi
2372,3147,"Green Mile, The (1999)",Crime|Drama
2690,3606,On the Town (1949),Comedy|Musical|Romance
3240,4380,"Princess and the Warrior, The (Krieger und die...",Drama|Romance
3461,4720,"Others, The (2001)",Drama|Horror|Mystery|Thriller
